In [1]:
import pandas as pd

In [5]:
club1 = pd.read_csv('../data/sample_video1/club/position_data.csv')
club2 = pd.read_csv('../data/sample_video2/club/position_data.csv')
conf1 = pd.read_csv('../data/sample_video1/club/confidence_data.csv')
conf2 = pd.read_csv('../data/sample_video2/club/confidence_data.csv')

In [8]:
club1.interpolate(method="linear", both=True, inplace=True)
club2.interpolate(method="linear", both=True, inplace=True)

IndentationError: unexpected indent (1248946786.py, line 2)